In [2]:
import numpy as np
import random 
from functools import reduce
from scipy.stats import unitary_group
from scipy.sparse import csr_matrix, kron
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image


In [3]:
# defining Pauli matrices & CNOTs
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

CNOT = [[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]] # control tensor switch
CNOT_12_34 = kron(CNOT, CNOT)

data_13_24 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
row_13_24 = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
column_13_24 = np.array([0,1,2,3,5,4,7,6,10,11,8,9,15,14,13,12])
CNOT_13_24 = csr_matrix((data_13_24,(row_13_24,column_13_24)), shape =(16,16))

data_14_23 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
row_14_23 = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
column_14_23 = np.array([0,1,2,3,6,7,4,5,9,8,11,10,15,14,13,12])
CNOT_14_23 = csr_matrix((data_14_23,(row_14_23,column_14_23)), shape =(16,16))

CNOTS = [CNOT_12_34,CNOT_13_24,CNOT_14_23]

# defining rotation gates
def Ry(A):
    sn, cs = np.sin(A/2), np.cos(A/2)
    M = np.array([[cs,-sn],[sn, cs]])
    return M
def Rz(A):
    phi = np.exp(-1j*A/2)
    M = np.array([[phi,0],[0,np.conj(phi)]])
    return M
    
#Returns a length-2 array instead of 2x2 matrix. Faster, since it's diagonal. Good for Z rotations!
def Rz2(A):
    phi = np.exp(-1j*A/2)
    return np.array([phi, np.conj(phi)])
    
def Rz2t(A):
    phi = np.exp(-1j*A/2)
    return np.array([[phi], [np.conj(phi)]])

Z2 = np.array([1,-1])
Z2t = np.array([[1],[-1]])
    
# Parameterising the general unitary with rotation gates
def U(A0,A1,A2): # Arguments: Euler Angles
    M = Rz2t(A1) * Ry(A0) * Rz2(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

#Compute U and all three derivatives together, to save computation
def DA_all(A0,A1,A2):
    R1 = Rz2t(A1)
    R0 = Ry(A0)
    R2 = Rz2(A2)
    R02 = R0 * R2
    R102 = R1 * R02
    U = R102
    dA0 = (-0.5j) * R1 * (Y @ R02)
    dA1 = (-0.5j) * Z2t * R102
    dA2 = (-0.5j) * R102 * Z2
    return (U, dA0, dA1, dA2)

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse
    
def parameters(layers): # layers = number of unitary layers, so no-of CNOTs = layers - 1
    A = np.zeros((4*layers,3))
    for i in range(4*layers):
        for j in range(3):
            A[i][j] = random.uniform(0.0, 4*np.pi) 
    return A
    
#Get the unitary created by these four single-qubit unitaries at layer i.
def U_4squ(A, i):
    U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
    U2 = U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
    if i%3== 0:
        return np.kron(np.kron(U1, U2), np.eye(4))
    if i%3== 1:
        return np.kron(np.kron(np.kron(np.eye(2), U1),U2),np.eye(2))
    if i%3== 2: 
        return np.kron(np.eye(4),np.kron(U1, U2))
    #return np.kron(np.kron(U1, U2), np.kron(U3, U4))

#Like `U_4squ` - But, optionally, skip the single unitary labelled j.
def U_4squ_skip(A, i, j):
    if i%3==0:
        if j%2==0:
            U1 = np.eye(2)
            U2 =  U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
        else:
            U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
            U2 = np.eye(2)
        return np.kron(np.kron(U1, U2), np.eye(4))
    if i%3==1:
        if j%2==0:
            U1 = np.eye(2)
            U2 =  U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
        else:
            U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
            U2 = np.eye(2)
        return np.kron(np.kron(np.kron(np.eye(2), U1),U2),np.eye(2))
    if i%3==2:
        if j%2==0:
            U1 = np.eye(2)
            U2 =  U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
        else:
            U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
            U2 = np.eye(2)
        return np.kron(np.eye(4),np.kron(U1, U2))
        
#Given a 1-qubit unitary Ui on site s (0-3), expand it to a 4-qubit unitary.
def expand_on_site(Ui, i, j): # i - layer, j - qubit number
    if i%3==0:
        if j%2==0:
            return np.kron(Ui, np.eye(8))
        else:
            return np.kron(np.eye(2), np.kron(Ui, np.eye(4)))
    if i%3==1:
        if j%2==0:
            return np.kron(np.eye(2), np.kron(Ui, np.eye(4)))
        else:
            return np.kron(np.eye(4), np.kron(Ui, np.eye(2)))
    if i%3==2:
        if j%2==0:
            return np.kron(np.eye(4), np.kron(Ui, np.eye(2)))
        else:
           return np.kron(np.eye(8), Ui) 
        

#Like `U_4squ`, but it's only the unitary at the one site j. That is, it's the single-qubit
#unitary, but expanded to 4-qubits.
def U_4squ_only(A, i, j):
    return expand_on_site(U(A[j][0],A[j][1],A[j][2]), i, j)

def circuit_structure(A, layers, j, U_goal): # qubit number    
    q = (j-1) // 4
    Uf = np.eye(16)
    Ub = np.eye(16)
    for it in range(q):
        Uf = Uf @ U_4squ(A, it) @ CNOTS[it%3]
    Uf = Uf @ U_4squ_skip(A, q, j-1)
    for ib in range(q+1, layers):
        Ub = Ub @ CNOTS[(ib-1)%3] @ U_4squ(A, ib)
    U_tbm = inverse(Uf) @ U_goal @ inverse(Ub) # U_tbm = rhs , U to be made
    return U_tbm

#The trace inner product of two matrices. Equal to np.trace(A @ inverse(B)), but faster
def trace_prod_inv(A, B):
    # return np.trace(A @ inverse(B))
    return np.conj(np.vdot(A, B))

In [4]:
# FUNCTIONS
# defining rotation gates
def Ry(A):
    sn, cs = np.sin(A/2), np.cos(A/2)
    M = np.array([[cs,-sn],[sn, cs]])
    return M
def Rz(A):
    phi = np.exp(-1j*A/2)
    M = np.array([[phi,0],[0,np.conj(phi)]])
    return M
    
# Returns a length-2 array instead of 2x2 matrix; faster, since it's diagonal and good for Z rotations!
def Rz2(A):
    phi = np.exp(-1j*A/2)
    return np.array([phi, np.conj(phi)])
    
def Rz2t(A):
    phi = np.exp(-1j*A/2)
    return np.array([[phi], [np.conj(phi)]])

Z2 = np.array([1,-1])
Z2t = np.array([[1],[-1]])
    
# Parameterising the general unitary with rotation gates
def U(A0,A1,A2): # Arguments: Euler Angles
    M = Rz2t(A1) * Ry(A0) * Rz2(A2)
    return M

# Compute U and all three derivatives together
def DA_all(A0,A1,A2):
    R1 = Rz2t(A1)
    R0 = Ry(A0)
    R2 = Rz2(A2)
    R02 = R0 * R2
    R102 = R1 * R02
    U = R102
    dA0 = (-0.5j) * R1 * (Y @ R02)
    dA1 = (-0.5j) * Z2t * R102
    dA2 = (-0.5j) * R102 * Z2
    return (U, dA0, dA1, dA2)

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse

# generating random initialization
def parameters(layers): 
    A = np.zeros((2*layers,3))
    for i in range(2*layers):
        for j in range(3):
            A[i][j] = random.uniform(0.0, 4*np.pi) 
    return A

# Get the unitary created by these four single-qubit unitaries at layer i.
def U_4squ(A, i):
    U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
    U2 = U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
    if i%3== 0:
        return np.kron(np.kron(U1, U2), np.eye(4))
    if i%3== 1:
        return np.kron(np.kron(np.kron(np.eye(2), U1),U2),np.eye(2))
    if i%3== 2: 
        return np.kron(np.eye(4),np.kron(U1, U2))
    #return np.kron(np.kron(U1, U2), np.kron(U3, U4))

# Like 'U_4squ' - But, optionally, skip the single unitary labelled j.
def U_4squ_skip(A, i, j):
    if i%3==0:
        if j%2==0:
            U1 = np.eye(2)
            U2 =  U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
        else:
            U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
            U2 = np.eye(2)
        return np.kron(np.kron(U1, U2), np.eye(4))
    if i%3==1:
        if j%2==0:
            U1 = np.eye(2)
            U2 =  U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
        else:
            U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
            U2 = np.eye(2)
        return np.kron(np.kron(np.kron(np.eye(2), U1),U2),np.eye(2))
    if i%3==2:
        if j%2==0:
            U1 = np.eye(2)
            U2 =  U(A[2*i+1][0],A[2*i+1][1],A[2*i+1][2])
        else:
            U1 = U(A[2*i  ][0],A[2*i  ][1],A[2*i  ][2])
            U2 = np.eye(2)
        return np.kron(np.eye(4),np.kron(U1, U2))
        
# Given a 1-qubit unitary Ui on site s (0-3), expand it to a 4-qubit unitary.
def expand_on_site(Ui, i, j): # i - layer, j - qubit number
    if i%3==0:
        if j%2==0:
            return np.kron(Ui, np.eye(8))
        else:
            return np.kron(np.eye(2), np.kron(Ui, np.eye(4)))
    if i%3==1:
        if j%2==0:
            return np.kron(np.eye(2), np.kron(Ui, np.eye(4)))
        else:
            return np.kron(np.eye(4), np.kron(Ui, np.eye(2)))
    if i%3==2:
        if j%2==0:
            return np.kron(np.eye(4), np.kron(Ui, np.eye(2)))
        else:
           return np.kron(np.eye(8), Ui) 
        

# Like `U_4squ`, but it's only the unitary at the one site j. That is, it's the single-qubit unitary, but expanded to 4-qubits.
def U_4squ_only(A, i, j):
    return expand_on_site(U(A[j][0],A[j][1],A[j][2]), i, j)

def circuit_structure(A, layers, j, U_goal): # qubit number    
    q = (j-1) // 2
    Uf = np.eye(16)
    Ub = np.eye(16)
    for it in range(q):
        Uf = Uf @ U_4squ(A, it) @ CNOTS[it%3]
    Uf = Uf @ U_4squ_skip(A, q, j-1)
    for ib in range(q+1, layers):
        Ub = Ub @ CNOTS[(ib-1)%3] @ U_4squ(A, ib)
    U_tbm = inverse(Uf) @ U_goal @ inverse(Ub) # U_tbm = rhs , U to be made
    return U_tbm

# The trace inner product of two matrices. Equal to np.trace(A @ inverse(B)), but faster
def trace_prod_inv(A, B):
    # return np.trace(A @ inverse(B))
    return np.conj(np.vdot(A, B))